In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
train_df  = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/train.csv')
test_df = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/test.csv')
movies_df = pd.read_csv('/kaggle/input/edsa-movie-recommender-challenge-2022/movies.csv')


In [4]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from sklearn.metrics import mean_squared_error
from math import sqrt
from surprise import SVD
import surprise
from surprise import Reader, Dataset
from sklearn.metrics.pairwise import cosine_similarity, pairwise_distances 


In [5]:
train_df.head()


In [6]:
test_df.head()

In [7]:
test_df.info()

In [8]:
submission_df = test_df.copy()
submission_df['Id'] = submission_df['userId'].map(str) +'_'+ submission_df['movieId'].map(str)
submission_df = submission_df.drop(['userId', 'movieId'], axis = 1)
submission_df

In [9]:
train_df.drop(columns=['timestamp'],inplace=True)

In [10]:
train_df.head(10)

In [11]:
train_df.info()

In [12]:
train_df.drop(train_df[train_df['userId'] >1000].index, inplace=True)
#sample = train_df.sample(n = 15000, random_state = 2021)

In [13]:
train_df

In [14]:
train_df.info()

In [15]:
train_df.sort_values('userId')

In [16]:
train_pivot = train_df.pivot(index='userId',columns='movieId',values='rating')
train_pivot

In [17]:
train_pivot.fillna(0,inplace=True)

In [18]:
train_pivot.head(10)

In [19]:
res = cosine_similarity(train_pivot)
res

In [20]:
similarity_table = pd.DataFrame(data = res, 
                               index = train_pivot.index,
                               columns = train_pivot.index)
                  
similarity_table

In [21]:
def movie_rating(reference_user, reference_movie):
    user_ref = similarity_table[reference_user]
    similiar_user = pd.DataFrame(user_ref)
    similiar_user = (similiar_user.sort_values(reference_user, ascending = False)).iloc[1:21]
    similiar_user.rename(columns={reference_user:'sim_to_current_user'},inplace=True)
    most_similiar_user_rating = train_pivot.filter([reference_movie])
    most_similiar_user_rating.rename(columns={reference_movie:'most_similiar_user_ratings_of_current_movie'},inplace=True)
    final_dataset = pd.merge(similiar_user, most_similiar_user_rating, on = 'userId')
    movie_rating = (final_dataset['sim_to_current_user'] * final_dataset['most_similiar_user_ratings_of_current_movie']).sum() / (final_dataset['sim_to_current_user']).sum()
    final_movie_rating = round(movie_rating*2)/2
    
    return final_movie_rating

In [24]:
movie_rating(803, 3822)

In [25]:
submission_df['rating'] = test_df.apply(lambda x: movie_rating(x['userId'], x['movieId']), axis=1)

In [26]:
submission_df.head(20)

In [ ]:
submission_df.to_csv(r'dconradie3.csv', index = False)